# DerivOmniModel Training - Google Colab

Training notebook for the multi-expert Deriv trading model.

## Setup
1. Upload your project to Google Drive
2. Upload your `data_cache/` folder with historical data
3. Run all cells

## 1. Mount Google Drive

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

## 2. Navigate to Project

In [ ]:
# Adjust this path to match your Drive folder structure
PROJECT_PATH = "/content/drive/MyDrive/xtitan"

import os

os.chdir(PROJECT_PATH)
print(f"Working directory: {os.getcwd()}")
!ls -la

## 3. Install Dependencies

In [ ]:
!pip install -q -r requirements-colab.txt

## 4. Check GPU

In [ ]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 5. Load Settings and Data

In [ ]:
import sys

sys.path.insert(0, PROJECT_PATH)

from dotenv import load_dotenv

load_dotenv()

from pathlib import Path

from config.settings import load_settings
from data.dataset import DerivDataset
from models.core import DerivOmniModel

settings = load_settings()
print(f"Settings loaded. Symbol: {settings.trading.symbol}")

In [ ]:
# Load dataset
data_path = Path("data_cache")
dataset = DerivDataset(data_path, settings, mode="train")
print(f"Dataset size: {len(dataset)} samples")

## 6. Train Model

In [ ]:
from torch.utils.data import DataLoader, random_split

from training.trainer import Trainer, TrainerConfig

# Train/val split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}")

In [ ]:
# DataLoaders
BATCH_SIZE = 128

train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True
)

val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True
)

In [ ]:
# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DerivOmniModel(settings)
print(f"Model parameters: {model.count_parameters():,}")

In [ ]:
# Training config
config = TrainerConfig(
    epochs=50,
    learning_rate=5e-4,
    checkpoint_dir=Path("checkpoints"),
    log_dir=Path("logs/tensorboard"),
    save_every=5,
    early_stop_patience=10,
)

trainer = Trainer(model, train_loader, val_loader, config, device)

In [ ]:
# Train!
results = trainer.train()

## 7. View Training Metrics

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/tensorboard

## 8. Test Model

In [ ]:
# Load best model
checkpoint = torch.load("checkpoints/best_model.pt")
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

# Test on validation sample
sample = next(iter(val_loader))
with torch.no_grad():
    probs = model.predict_probs(
        sample["ticks"].to(device), sample["candles"].to(device), sample["vol_metrics"].to(device)
    )

print("Sample predictions:")
for k, v in probs.items():
    print(f"  {k}: {v[:5].tolist()}")

## 9. Save to Drive

In [ ]:
# Checkpoints are already in Drive since we're working directly from it
print(f"Best model saved: {results['best_checkpoint']}")
print(f"Best validation loss: {results['best_val_loss']:.4f}")
print(f"Epochs trained: {results['epochs_trained']}")